In [2]:
import pandas as pd
import networkx as nx
import openpyxl
import matplotlib.pyplot as plt

In [4]:
# 単価テーブル
unit_price = pd.read_excel('ブレーカ調査/CT単価表.xlsx', sheet_name='Sheet1')

In [6]:
# 案1_30A以上
case_1 = pd.read_excel('ブレーカ調査/案1_30A以上.xlsx', sheet_name='Sheet1')

In [7]:
# 必要CT数
line_num = pd.read_excel('ブレーカ調査/必要CT数.xlsx', sheet_name='Sheet1')

必要CT数とCT代計算

In [52]:
for index, row in case_1.iterrows():
    if row['電力量計\n取付対象']==1:
        volt_str = row['電圧\n200/100']
        try:
            volt = int(volt_str)
        except:
            if volt_str == '200/100':
                volt = 200
            else:
                volt = None
        phase_str = row['相\n3/1']
        try:
            phase = int(phase_str)
        except:
            phase = None
        current_str = row['ﾌﾞﾚｰｶ容量\nA']
        try:
            current = int(current_str)
        except:
            current = None
        diameter_str = row['ｹｰﾌﾞﾙ径\nmm']
        try:
            diameter = int(diameter_str)
        except:
            diameter = None
        line = int(row['線'])
        
        filtered = unit_price[(unit_price['最大線径[mm]'] >= diameter) & (unit_price['1次側定格電流[A]'] >= current)]
        if not filtered.empty:
            case_1.at[index,'型番'] = filtered.iloc[0]['型番']
            case_1.at[index,'単価'] = filtered.iloc[0]['単価']
        else:
            print('Not Found:', row['No.'],diameter,current)
            case_1.at[index,'型番'] = None
            case_1.at[index,'単価'] = None
        filtered_ct_num = line_num[(line_num['相'] == phase) & (line_num['線'] == line)]
        if not filtered_ct_num.empty:
            case_1.at[index,'必要CT数'] = filtered_ct_num.iloc[0]['必要CT数']
        else:
            print('Not Found:', row['No.'],phase,line)
            case_1.at[index,'必要CT数'] = None
case_1['小計'] = case_1['単価'] * case_1['必要CT数']



CTはそろったので、必要な電力計数を算出したい

In [58]:
case_1.columns

Index(['No.', '建屋', '階', '部屋名', '部署', '分電盤\nNo.', '分電盤名', '親分電盤\n(あれば)',
       '電力量計\nあり/なし', 'ﾌﾞﾚｰｶ番号\n(あれば)', '電圧\n200/100', '相\n3/1', 'ﾌﾞﾚｰｶ容量\nA',
       '負荷名', 'ｹｰﾌﾞﾙ径\nmm', '備考', '電力量計\n取付対象', '線', '型番', '単価', '必要CT数', '小計',
       '電力計', '電力計単価', 'M5Stack', 'M5Stack単価'],
      dtype='object')

In [59]:
# 電力計の連番を振るためのカウンタ
power_meter_counter = 1

# 電力計の連番を格納するリスト
power_meter_list = []

panel_name_unique = case_1['分電盤名'].unique()

# 分電盤ごとに処理を行う
for panel in panel_name_unique:
    print(panel)
    panel_data = case_1[case_1['分電盤名'] == panel].reset_index(drop=True)
    ct_count = 0
    for index, row in panel_data.iterrows():
        if pd.notnull(row['必要CT数']):
            ct_count += row['必要CT数']
            if ct_count <= 4:
                power_meter_list.append(f'電力計_{power_meter_counter}')
            else:
                power_meter_counter += 1
                ct_count = row['必要CT数']
                power_meter_list.append(f'電力計_{power_meter_counter}')
        else:
            power_meter_list.append(None)
    if panel_data['電力量計\n取付対象'].sum() > 0:
        power_meter_counter += 1

    if power_meter_counter == 37:
        print('HIT')
    print(power_meter_counter)

# 電力計の連番をデータフレームに追加
case_1['電力計'] = power_meter_list


L33-1A
2
M33-1A
7
M33-1W
12
M33-1W-1盤
14
排気ﾌｧﾝ操作盤
14
ﾎﾙﾀﾞ測定室分電盤
14
M33-1H(左)
16
電源盤
18
動力盤B
20
動力盤C
21
M33-1H(右)
22
M33-1I
24
ﾌｧﾝ制御盤
26
M33-1G
27
M33-1J
28
M33-1K
33
M33-1K-1
35
空調ｴｱﾊﾝ運転盤
36
工具1F動力M-1(左)
36
工具1F動力M-2(中)
36
工具1F動力M-3(右)
36
M33-1J-1(1)
40
恒温ﾌﾞｰｽ用動力盤
42
M33-1J-1(2)
47
M33-1B
51
M33-1B-1
53
L33-1B
54
動力盤D
55
M33-1L
58
M33-1N(A)
59
M33-1N(B)
62
M33-1F
64
1L-1
65
1M-1
69
動力盤1M-1-1
71
L33-2A(左)
72
L33-2A(右)
72
M33-2A
73
L33-2E
74
M33-2F
95
洗浄機盤
96
局所排気設備制御盤電源盤
96
局所排気設備制御盤
98
ﾀﾞｸﾄﾌｧﾝ操作盤
98
ﾜｲﾔｰ加工電源盤
102
排熱ﾀﾞｸﾄ盤
102
PAC-1
103
動力分電盤
104
2L-1
105
L33-3A
107
M33-3A
108
L33-3(検査)
109
M33-31(NT)
112
M33-3(再研)
113
M33-3-1(再研)
113
3L-1
114
M33-3B
116
ML33-4A
117
M33-4D
119
工具EHP→工具空調機動力盤
122
工具電灯関係
122
熱源制御盤
122
L-1南側外灯電源盤
123
局所排気装置動力盤
124
空調動力盤
124
工具ﾚｲｱｳﾄA盤
125
工具ﾚｲｱｳﾄB盤
128
工具M-1
141
工具FAN制御盤
142
盤名なし
143
工具M-1-2
146
工具L-1
147
ML-30
151
局所排気制御盤
152
ML-2
159
動力盤A
160
工具L-2
161
SE-3 B盤
163
SE-3 A盤
169
手元開閉盤
169
電灯盤L-2
170
ﾚｰｽｳｪｲ分電盤
171
電灯盤ML-B.C.D
173


In [16]:
del case_1['電力計単価']

In [18]:
filled = []
for index,row in case_1.iterrows():
    if pd.notnull(row['電力計']):
        if not row['電力計'] in filled:
            filled.append(row['電力計'])
            case_1.at[index,'電力計単価'] = 11640


In [20]:
int(case_1['小計'].sum())

2092820

In [21]:
int(case_1['電力計単価'].sum())

2002080

4094900

In [60]:
# M5Stackの連番を振るためのカウンタ
m5stack_counter = 1

# M5Stackの連番を格納するリスト
m5stack_list = []

# 分電盤ごとに処理を行う
for panel in case_1['分電盤名'].unique():
    panel_data = case_1[case_1['分電盤名'] == panel]
    power_meter_count = 0
    for index, row in panel_data.iterrows():
        if pd.notnull(row['電力計']):
            power_meter_count += 1
            if power_meter_count > 2:
                m5stack_counter += 1
                power_meter_count = 1
            m5stack_list.append(f'M5Stack_{m5stack_counter}')
        else:
            m5stack_list.append(None)
    if panel_data['電力量計\n取付対象'].sum() > 0:
        m5stack_counter += 1

# M5Stackの連番をデータフレームに追加
case_1['M5Stack'] = m5stack_list

In [61]:
filled = []
for index,row in case_1.iterrows():
    if pd.notnull(row['M5Stack']):
        if not row['M5Stack'] in filled:
            filled.append(row['M5Stack'])
            case_1.at[index,'M5Stack単価'] = 7000

In [65]:
int(case_1['小計'].sum())

2092820

In [63]:
int(case_1['電力計単価'].sum())

2002080

In [64]:
int(case_1['M5Stack単価'].sum())

1211000

In [32]:
case_1.to_excel('ブレーカ調査/案1_30A以上_計算結果.xlsx', index=False)

In [66]:
# 型番ごとの必要数をカウント
model_counts = case_1['型番'].value_counts()

# 結果を表示
print(model_counts)
# 型番ごとの必要数量を計算
model_total_counts = case_1.groupby('型番')['必要CT数'].sum()

# 結果を表示
print(model_total_counts)

型番
M-NCT-5A/50A    147
KM-NCT-225A      76
KM-NCT-100A      63
KM-NCT-400A      25
KM-NCT-600A       1
Name: count, dtype: int64
型番
KM-NCT-100A     125.0
KM-NCT-225A     152.0
KM-NCT-400A      49.0
KM-NCT-600A       2.0
M-NCT-5A/50A    289.0
Name: 必要CT数, dtype: float64


In [69]:
case_1['型番'][case_1['型番'] == 'M-NCT-5A/50A'].count()

np.int64(147)

In [71]:
case_1['型番'].unique()

array(['KM-NCT-100A', nan, 'KM-NCT-225A', 'M-NCT-5A/50A', 'KM-NCT-400A',
       'KM-NCT-600A'], dtype=object)

In [72]:
result =[]
for model in case_1['型番'].unique():
    print(model)
    print(case_1['必要CT数'][case_1['型番'] == model].sum())
    result.append([model,case_1['必要CT数'][case_1['型番'] == model].sum()])

KM-NCT-100A
125.0
nan
0.0
KM-NCT-225A
152.0
M-NCT-5A/50A
289.0
KM-NCT-400A
49.0
KM-NCT-600A
2.0


In [74]:
df = pd.DataFrame(result)

In [76]:
case_1.to_excel('ブレーカ調査/案1_30A以上_計算結果_241205.xlsx', index=False)